In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
%matplotlib inline
from torchvision.models import resnet18, ResNet18_Weights

# Set the device
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")


Using device: mps


In [2]:
#!pip install tensorboard
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("./logs")

In [7]:
#CIFAR-10 dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
batch_size = 32
# Downloading/Loading CIFAR10 data
trainset = torchvision.datasets.CIFAR10(root='./CIFAR10', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./CIFAR10', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size*4, shuffle=False, num_workers=2)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [4]:
from tqdm import tqdm
import torch.optim as optim
import time

In [13]:
#Create model ResNet19
#Add fc layer to pretrained ResNet18(18 + 1)

In [14]:
target = len(classes)

resnet19 = resnet18(weights=ResNet18_Weights.DEFAULT)

num_ftrs = resnet19.fc.in_features
resnet19.fc = nn.Sequential(nn.Linear(num_ftrs, num_ftrs//2),nn.ReLU(), nn.Dropout(p=0.2), nn.Linear(num_ftrs//2, target))

print(resnet19)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet19.parameters(), lr = 0.001, momentum = 0.9)
len(trainloader)
resnet19.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
# Number of training epochs
num_epochs = 1
# Training loop

for epoch in range(num_epochs):
    resnet19.train()
    loop = tqdm(trainloader)
    eval_loop = tqdm(testloader)
    running_loss = 0.0
    start_time = time.time()

    for i, (inputs, labels) in enumerate(loop, 1):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = resnet19(inputs)
        loss = criterion(outputs, labels)

        # Backpropagation and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        loop.set_postfix(loss = loss.item())
    end_time = time.time()
    print(f"Epoch {epoch + 1} took {end_time - start_time:.2f} seconds")
    print(f"[{epoch + 1}, {i + 1:5d}] loss: {running_loss / i:.3f}")
    
    #eval step
    correct = 0
    total = 0
    resnet19.eval()
    # Disable gradient computation for evaluation
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(eval_loop, 1):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = resnet19(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Accuracy on the test set: {100 * correct / total:.2f}%")


print("Finished Training")


In [8]:
#Create model ResNet19 2
#Add conv layer to pretrained ResNet18

In [5]:
resnet19_conv = resnet18(weights=ResNet18_Weights.DEFAULT)

additional_conv_layer = nn.Sequential(
    nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=False),
    nn.BatchNorm2d(512),
    nn.ReLU(inplace=True))
resnet19_conv.layer4.add_module("additional_conv_layer", additional_conv_layer)

num_ftrs = resnet19_conv.fc.in_features
resnet19_conv.fc = nn.Linear(num_ftrs, target)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet19_conv.parameters(), lr = 0.001, momentum = 0.9)

resnet19_conv.to(device)



ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [10]:
# Number of training epochs
num_epochs = 1
# Training loop

for epoch in range(num_epochs):
    resnet19_conv.train()
    loop = tqdm(trainloader)
    eval_loop = tqdm(testloader)
    running_loss = 0.0
    start_time = time.time()

    for i, (inputs, labels) in enumerate(loop, 1):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = resnet19_conv(inputs)
        loss = criterion(outputs, labels)

        # Backpropagation and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        loop.set_postfix(loss = loss.item())
    end_time = time.time()
    print(f"Epoch {epoch + 1} took {end_time - start_time:.2f} seconds")
    print(f"[{epoch + 1}, {i + 1:5d}] loss: {running_loss / i:.3f}")
    
    #eval step
    correct = 0
    total = 0
    resnet19_conv.eval()
    # Disable gradient computation for evaluation
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(eval_loop, 1):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = resnet19_conv(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Accuracy on the test set: {100 * correct / total:.2f}%")


print("Finished Training")

100%|████████████████████████████| 12500/12500 [06:37<00:00, 31.44it/s, loss=2.01]

Epoch 1 took 397.60 seconds
[1, 12501] loss: 2.100




100%|█████████████████████████████████████████| 2500/2500 [07:10<00:00,  5.80it/s]

Accuracy on the test set: 41.02%
Finished Training


In [ ]:
#Test 3 structure like https://arxiv.org/pdf/2011.05280.pdf

In [31]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, padding=0),
                nn.BatchNorm2d(out_channels)
            )
        self.relu_out = nn.ReLU()

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        out += self.shortcut(x)
        out = self.relu_out(out)
        return out

class ResNet19_SNN(nn.Module):
    def __init__(self):
        super(ResNet19_SNN, self).__init__()

        self.conv1 = nn.Conv2d(3, 128, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(128)
        self.relu = nn.ReLU()

        self.block1 = nn.Sequential(
            BasicBlock(128, 128, stride=1),
            BasicBlock(128, 128, stride=1),
            BasicBlock(128, 128, stride=1)
        )

        self.block2 = nn.Sequential(
            BasicBlock(128, 256, stride=2),
            BasicBlock(256, 256, stride=1),
            BasicBlock(256, 256, stride=1),
        )

        self.block3 = nn.Sequential(
            BasicBlock(256, 512, stride=2),
            BasicBlock(512, 512, stride=1)
        )

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(512, 256)
        self.relu2 = nn.ReLU()
        self.fc2 = nn.Linear(256, 10)
        

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)

        x = self.avgpool(x)
        #print("dopo pool", x.size())
        x = x.view(x.size(0), -1)
        #print("wiew pool", x.size())
        x = self.fc1(x)
        x = self.relu2(x)
        x = self.fc2(x)
        
        return x
ResNet19_SNN_model = ResNet19_SNN()
ResNet19_SNN_model.to(device)

ResNet19_SNN(
  (conv1): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (block1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
      (relu_out): ReLU()
    )
    (1): BasicBlock(
      (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): Batch

In [32]:
import torch.nn.init as init

def initialize_weights(m):
    if isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
        if m.bias is not None:
            init.constant_(m.bias, 0)
    elif isinstance(m, nn.BatchNorm2d):
        init.constant_(m.weight, 1)
        init.constant_(m.bias, 0)
    elif isinstance(m, nn.Linear):
        init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
        init.constant_(m.bias, 0)

In [36]:
import torch.optim.lr_scheduler as ls


#ResNet19_SNN_model.apply(initialize_weights)
criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(ResNet19_SNN_model.parameters(), lr = 0.01)
optimizer = optim.SGD(ResNet19_SNN_model.parameters(), lr = 0.01, weight_decay = 0.01)
#adam / dro
scheduler = ls.StepLR(optimizer, step_size=30, gamma=0.3)
#https://medium.com/analytics-vidhya/deep-learning-basics-weight-decay-3c68eb4344e9

# Number of training epochs
num_epochs = 50
# Training loop

for epoch in range(num_epochs):
    ResNet19_SNN_model.train()
    loop = tqdm(trainloader)
    eval_loop = tqdm(testloader)
    running_loss = 0.0
    start_time = time.time()
    correct = 0
    total = 0
    
    for i, (inputs, labels) in enumerate(loop, 1):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = ResNet19_SNN_model(inputs)
        loss = criterion(outputs, labels)

        # Backpropagation and optimization
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        running_loss += loss.item()
        loop.set_postfix(loss = loss.item())
    end_time = time.time()
    scheduler.step()
    print(f"Epoch {epoch + 1} took {end_time - start_time:.2f} seconds")
    print(f"[{epoch + 1}, {i + 1:5d}] loss: {running_loss / i:.3f}")
    print(f"lr: {scheduler.get_last_lr()[0]}")
    print(f"Accuracy on the test set: {100 * correct / total:.2f}%")
    writer.add_scalar("LOSS/Training",loss.item(), epoch)
    writer.add_scalar("LR/Training",scheduler.get_last_lr()[0], epoch)
    writer.add_scalar("ACCURACY/Training", correct / total, epoch)
    
    #eval step
    correct = 0
    total = 0
    ResNet19_SNN_model.eval()
    # Disable gradient computation for evaluation
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(eval_loop, 1):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = ResNet19_SNN_model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Accuracy on the test set: {100 * correct / total:.2f}%")
    writer.add_scalar("ACCURACY/Evaluation",(correct / total), epoch) 
print("Finished Training")

  0%|                                                    | 0/1563 [00:00<?, ?it/s]

100%|█████████████████████████████| 1563/1563 [07:39<00:00,  3.40it/s, loss=0.876]

Epoch 1 took 459.85 seconds
[1,  1564] loss: 0.888
lr: 0.01
Accuracy on the test set: 68.50%





  1%|▌                                         | 1/79 [07:43<10:01:55, 463.01s/it]

  3%|█                                          | 2/79 [07:43<4:04:53, 190.82s/it]

  4%|█▋                                         | 3/79 [07:43<2:11:29, 103.81s/it]

  5%|██▏                                         | 4/79 [07:43<1:18:40, 62.94s/it]

  6%|██▉                                           | 5/79 [07:44<49:45, 40.34s/it]

  8%|███▍                                          | 6/79 [07:44<32:30, 26.72s/it]

  9%|████                                          | 7/79 [07:44<21:41, 18.08s/it]

 10%|████▋                                         | 8/79 [07:44<14:41, 12.41s/it]

 11%|█████▏                                        | 9/79 [07:45<10:03,  8.62s/it]

 13%|█████▋                                       | 10/79 [07:45<06:57,  6.04s/it]

 14%|██████▎                                      | 11/79 [07:45<04:50,  4.28s/it]

 15%|██████▊                                      | 12/79 [07:46<03:25,  

Accuracy on the test set: 56.05%


100%|█████████████████████████████| 1563/1563 [08:32<00:00,  3.05it/s, loss=0.589]

...

Accuracy on the test set: 85.05%


100%|████████████████████████████| 1563/1563 [08:04<00:00,  3.23it/s, loss=0.0477]

Epoch 50 took 484.31 seconds
[50,  1564] loss: 0.056
lr: 0.003
Accuracy on the test set: 99.19%




100%|█████████████████████████████████████████████| 79/79 [08:37<00:00,  6.55s/it]

Accuracy on the test set: 85.77%
Finished Training
